In [1]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.applications import EfficientNetB0, InceptionV3, VGG16, EfficientNetB1, EfficientNetB2,EfficientNetB3


In [2]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    else:
        raise ValueError("Escala de cor não suportada.")

In [3]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [4]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

            # Converta para a escala de cor desejada
            img = convert_color_scale(img, color_scale)

            # Se a imagem estiver em escala de cinza, expanda as dimensões
            if color_scale == 'grayscale':
                img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

            images.append(img)
    return np.array(images)

In [5]:
# Defina as pastas de dados
data_dir = r"C:\Users\joaob\Jupyter Notebooks\HPV\images\40X"
normal_dir = os.path.join(data_dir, 'nao_tumor')
cancer_dir = os.path.join(data_dir, 'tumor')
img_extensions = ['.jpg', '.jpeg', '.png']

In [6]:
def execute_model_by_color(chosen_color_scale): # Escolha a escala de cor desejada # Pode ser 'hsv', 'rgb', 'grayscale', 'lab', 'luv'
   

    # Carregamento de imagens com a escala de cor escolhida
    normal_images = load_images(normal_dir, chosen_color_scale)
    cancer_images = load_images(cancer_dir, chosen_color_scale)

    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])
    

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    # Escolha entre 5 tipos de conversão de escala de cor aqui (por exemplo, normalização)
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Definir modelo VGG-16
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compilar o modelo
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Configurar o retorno de chamada ModelCheckpoint
    checkpoint = callbacks.ModelCheckpoint("best_model.h5",
                                           monitor='val_accuracy',  # Métrica a ser monitorada
                                           save_best_only=True,      # Salvar apenas o melhor modelo
                                           mode='max',               # Salvar o modelo com a maior precisão
                                           verbose=1)
    # Treinar o modelo
    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

    # Avaliar o modelo
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)
    metrics = get_metrics(y_test, y_pred_binary)
    
    # Excluir o modelo atual para liberar memória da GPU   
    del all_images, normal_images, cancer_images, X_train, X_test, y_train, y_test
    del all_labels, normal_labels, cancer_labels
    del model, base_model
    gc.collect()
    
    return metrics

In [ ]:
chosen_color_scale = "xyz"
metrics = execute_model_by_color(chosen_color_scale)
df_metrics_xyz = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_xyz.insert(0, 'Classificador', chosen_color_scale.upper())

chosen_color_scale = "hsv"
metrics = execute_model_by_color("hsv")
df_metrics_hsv = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_hsv.insert(0, 'Classificador', chosen_color_scale.upper())

chosen_color_scale = "luv"
metrics = execute_model_by_color(chosen_color_scale)
df_metrics_luv = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_luv.insert(0, 'Classificador', chosen_color_scale.upper())

chosen_color_scale = "rgb"
metrics = execute_model_by_color(chosen_color_scale)
df_metrics_rgb = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_rgb.insert(0, 'Classificador', chosen_color_scale.upper())

chosen_color_scale = "grayscale"
metrics = execute_model_by_color(chosen_color_scale)
df_metrics_gray = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_gray.insert(0, 'Classificador', chosen_color_scale.upper())

chosen_color_scale = "lab"
metrics = execute_model_by_color(chosen_color_scale)
df_metrics_lab = pd.DataFrame([metrics.values()], columns=metrics.keys())
df_metrics_lab.insert(0, 'Classificador', chosen_color_scale.upper())

df_all_metrics = pd.concat([df_metrics_xyz, df_metrics_hsv, df_metrics_luv, df_metrics_rgb, df_metrics_gray, df_metrics_lab])

print("Métricas para seleção do melhor esquema de cor: ")
df_all_metrics